# Pràctica  Fifa - Joan Alcover Lladó

L'objectiu d'aquesta pràctica és preveure el *valor* dels jugadors utilitzant la seva informació.

## Importació de les llibreries

El primer de tot és importar les llibreries que utilitzarem en aquesta pràctica.

In [1]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np

# Lectura de les dades

Per llegir les dades utilitzarem ``pandas`` i una utilitat de la llibreria ``os``.


In [2]:
df = pd.read_csv(os.path.join("..", "in", "fifa.csv"))
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


## Anàlisi de dades 

En primer lloc volem saber què passa a les dades. Per fer això el que feim és incrementar el nombre de columnes a mostrar. 


In [3]:
pd.set_option('display.max_columns', 13)
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,...,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,...,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,...,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,...,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,...,15.0,13.0,5.0,10.0,13.0,€196.4M


## Columnes innecessàries

Observant les dades hem vist que hi ha columnes innecessàries, és a dir, que no ens aportaran cap valor a l'hora de fer la previsió desitjada.

Aquestes columnes són les que eliminarem a continuació:

In [4]:
df.drop(columns=['Unnamed: 0','ID','Name','Photo','Flag','Club Logo','Special','Preferred Foot','Real Face','Jersey Number','Joined','Loaned From','Contract Valid Until','LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM', 'LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB','CB', 'RCB','RB','Body Type'], axis='columns', inplace=True)
df.head()

,Age,Nationality,Overall,Potential,Club,Value,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,...,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,...,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,...,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,...,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,...,15.0,13.0,5.0,10.0,13.0,€196.4M


## Tractament dels NaN

Ara el que ens interessa és tractar els valors NaN que ens surten a les dades.

El primer que farem és visualitzar quines columnes contenen NaNs per veure com podrem solucionar-ho.

In [5]:
df.columns[df.isna().any()].tolist()

['Club',
 'International Reputation',
 'Weak Foot',
 'Skill Moves',
 'Work Rate',
 'Position',
 'Height',
 'Weight',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes',
 'Release Clause']

Ara veurem el nombre de NaNs per columna.

In [6]:
pd.set_option('display.max_rows', None)
df.isnull().sum()

Age                            0
Nationality                    0
Overall                        0
Potential                      0
Club                         241
Value                          0
Wage                           0
International Reputation      48
Weak Foot                     48
Skill Moves                   48
Work Rate                     48
Position                      60
Height                        48
Weight                        48
Crossing                      48
Finishing                     48
HeadingAccuracy               48
ShortPassing                  48
Volleys                       48
Dribbling                     48
Curve                         48
FKAccuracy                    48
LongPassing                   48
BallControl                   48
Acceleration                  48
SprintSpeed                   48
Agility                       48
Reactions                     48
Balance                       48
ShotPower                     48
Jumping   

In [7]:
pd.set_option('display.max_rows', 8)

Podem observar que hi ha moltes columnes amb un nombre de NaNs relativament baix (<250) en comparació a totes les dades que tenim. 

Degut a aquest baix percentatge respecte al conjunt de les dades procedirem a eliminar aquests jugadors.

In [8]:
df = df.dropna(subset = ['Club','International Reputation','Weak Foot','Skill Moves','Work Rate','Position','Height','Weight','Crossing','Finishing', 'HeadingAccuracy','ShortPassing','Volleys','Dribbling','Curve', 'FKAccuracy','LongPassing','BallControl','Acceleration','SprintSpeed', 'Agility','Reactions','Balance','ShotPower','Jumping', 'Stamina','Strength','LongShots','Aggression','Interceptions', 'Positioning','Vision','Penalties','Composure', 'Marking','StandingTackle','SlidingTackle', 'GKDiving','GKHandling','GKKicking','GKPositioning','GKReflexes'])
df

,Age,Nationality,Overall,Potential,Club,Value,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,...,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,...,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,...,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,...,90.0,85.0,87.0,88.0,94.0,€138.6M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18203,19,Sweden,47,63,Trelleborgs FF,€60K,...,10.0,9.0,9.0,5.0,12.0,€113K
18204,16,England,47,67,Cambridge United,€60K,...,6.0,5.0,10.0,6.0,13.0,€165K
18205,17,England,47,66,Tranmere Rovers,€60K,...,14.0,6.0,14.0,8.0,9.0,€143K
18206,16,England,46,66,Tranmere Rovers,€60K,...,10.0,15.0,9.0,12.0,9.0,€165K


In [9]:
df[df.isna().any(axis=1)]

,Age,Nationality,Overall,Potential,Club,Value,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
28,26,Colombia,88,89,FC Bayern München,€69.5M,...,15.0,15.0,15.0,5.0,14.0,NaN
38,30,Argentina,88,88,Milan,€57M,...,5.0,12.0,7.0,5.0,10.0,NaN
91,29,Brazil,85,85,Guangzhou Evergrande Taobao FC,€37M,...,16.0,16.0,11.0,6.0,8.0,NaN
166,24,Brazil,83,90,Guangzhou Evergrande Taobao FC,€36.5M,...,13.0,11.0,13.0,12.0,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18026,21,China PR,50,59,Guizhou Hengfeng FC,€50K,...,15.0,8.0,10.0,13.0,7.0,NaN
18031,20,China PR,50,61,Stabæk Fotball,€40K,...,11.0,7.0,9.0,10.0,6.0,NaN
18056,19,Italy,50,65,Ascoli,€60K,...,8.0,11.0,13.0,14.0,14.0,NaN
18183,44,England,48,48,Cambridge United,€0,...,45.0,48.0,44.0,49.0,46.0,NaN


Encara ens falten NaNs per tractar.

Observam que els que ens falten son els de las columna **Release Clause**. Podem suposar que si no tenen cap clàusula vol dir que aquesta mateixa val 0€, per tant substituirem els NaN per €0.

In [10]:
df = df.fillna('€0')
df.head()

,Age,Nationality,Overall,Potential,Club,Value,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,...,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,...,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,...,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,...,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,...,15.0,13.0,5.0,10.0,13.0,€196.4M


Tot i que abans haguem eliminat els jugadors que no tenien club encara tenim un problema i és que de la manera en que està el **Club** no el podrem tractar. Per solucionar això utilitzarem One-Hot Encoding i, d'aquesta manera, transformarem el Club en int afegint tantes columnes com clubs hi hagi. Si un jugador pertany a un club determinat, aquest jugador tendrà un valor 1 a la columna del club al qual pertany, si no tendrà valor 0.

In [11]:
clb = df.pop("Club")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(clb, prefix='clb').reset_index(drop=True)], axis=1, sort=False)
df.head()

,Age,Nationality,Overall,Potential,Value,Wage,...,clb_Zagłębie Lubin,clb_Zagłębie Sosnowiec,clb_Çaykur Rizespor,clb_Örebro SK,clb_Östersunds FK,clb_Śląsk Wrocław
0,31,Argentina,94,94,€110.5M,€565K,...,0,0,0,0,0,0
1,33,Portugal,94,94,€77M,€405K,...,0,0,0,0,0,0
2,26,Brazil,92,93,€118.5M,€290K,...,0,0,0,0,0,0
3,27,Spain,91,93,€72M,€260K,...,0,0,0,0,0,0
4,27,Belgium,91,92,€102M,€355K,...,0,0,0,0,0,0


Feim el mateix per a les columnes **Nationality**, **Work Rate**, **Body Type** i **Position**.

In [12]:
nty = df.pop("Nationality")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(nty, prefix='nty').reset_index(drop=True)], axis=1, sort=False)

wr = df.pop("Work Rate")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(wr, prefix='wr').reset_index(drop=True)], axis=1, sort=False)

pos = df.pop("Position")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(pos, prefix='pos').reset_index(drop=True)], axis=1, sort=False)

df.head()

,Age,Overall,Potential,Value,Wage,International Reputation,...,pos_RF,pos_RM,pos_RS,pos_RW,pos_RWB,pos_ST
0,31,94,94,€110.5M,€565K,5.0,...,1,0,0,0,0,0
1,33,94,94,€77M,€405K,5.0,...,0,0,0,0,0,1
2,26,92,93,€118.5M,€290K,5.0,...,0,0,0,0,0,0
3,27,91,93,€72M,€260K,4.0,...,0,0,0,0,0,0
4,27,91,92,€102M,€355K,4.0,...,0,0,0,0,0,0


Comprovam que no tenim cap NaN a les dades

In [13]:
df[df.isna().any(axis=1)]

,Age,Overall,Potential,Value,Wage,International Reputation,...,pos_RF,pos_RM,pos_RS,pos_RW,pos_RWB,pos_ST


Amb tot això ja hem solucionat els problemes dels NaNs que ens sortien a les dades.

## Tractament de les dades

Ara que hem arreglat el problema dels NaNs tenim un altre problema, hi ha valors que no són completament numèrics com per exemple les columnes **Value** i **Wage** d'entre altres.

Dit això procedirem a tractar aquestes dades definint les següents funcions:

In [14]:
def value_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

In [15]:
def foot_to_float(x):
    """
    From Foot to float.
    
    """
    x = float(x.replace("'", '.'))
    return x

In [16]:
def lbs_to_float(x):
    """
    From lbs to float.
    
    """
    x = float(x.replace('lbs', ''))
    return x

Ara aplicarem aquestes funcions a les columnes corresponents.

In [17]:
df["Value"] = df["Value"].apply(value_to_float)
df["Wage"] = df["Wage"].apply(value_to_float)
df["Release Clause"] = df["Release Clause"].apply(value_to_float)

In [18]:
df["Height"] = df["Height"].apply(foot_to_float)

In [19]:
df["Weight"] = df["Weight"].apply(lbs_to_float)
df.head()

,Age,Overall,Potential,Value,Wage,International Reputation,...,pos_RF,pos_RM,pos_RS,pos_RW,pos_RWB,pos_ST
0,31,94,94,110500000.0,565000.0,5.0,...,1,0,0,0,0,0
1,33,94,94,77000000.0,405000.0,5.0,...,0,0,0,0,0,1
2,26,92,93,118500000.0,290000.0,5.0,...,0,0,0,0,0,0
3,27,91,93,72000000.0,260000.0,4.0,...,0,0,0,0,0,0
4,27,91,92,102000000.0,355000.0,4.0,...,0,0,0,0,0,0


Fet això ja tenim totes les dades preparades per al següent pas.

## Predicció

Finalment procedirem a realitzar la predicció del valor del jugador amb les dades completament numèriques que hem treballat anteriorment.

In [20]:
val = df.pop("Value")
df

,Age,Overall,Potential,Wage,International Reputation,Weak Foot,...,pos_RF,pos_RM,pos_RS,pos_RW,pos_RWB,pos_ST
0,31,94,94,565000.0,5.0,4.0,...,1,0,0,0,0,0
1,33,94,94,405000.0,5.0,4.0,...,0,0,0,0,0,1
2,26,92,93,290000.0,5.0,5.0,...,0,0,0,0,0,0
3,27,91,93,260000.0,4.0,3.0,...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17914,19,47,63,1000.0,1.0,2.0,...,0,0,0,0,0,1
17915,16,47,67,1000.0,1.0,3.0,...,0,0,0,0,0,1
17916,17,47,66,1000.0,1.0,3.0,...,0,0,0,1,0,0
17917,16,46,66,1000.0,1.0,3.0,...,0,0,0,0,0,0


In [21]:
X_train, X_test, y_train, y_test = train_test_split(df, val, test_size=0.33, random_state=42)

In [22]:
len(X_train)

12005

Ara entrenarem un model de Regressió Lineal.

In [23]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

Finalment obtenim una mètrica $R^2$ per a la regressió, utilitzam la implementació de sickit-learn.

In [24]:
preds = reg.predict(X_test)

In [25]:
preds[0]

-364695.48428690946

In [26]:
y_test[0]

110500000.0

In [27]:
r2_score(preds, y_test)

0.9660008433165387

El resultat és aproximadament 0,966, el qual és un nombre bastant proper a 1 i per tant podem concloure que la nostra predicció és acertada.